In [1]:
import duckdb

Setup File Path & Object Name di MinIO

In [2]:
localPath = {
    'jakarta': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah/cleanedData/dapodik/dataYangSudahDiKategorisasi.csv',
    'bekasi': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Bekasi.csv',
    'depok': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Depok.csv',
    'balikpapan': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/bekasiMatchCategorized.csv',
    'makassar': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/makassarMatchCategorized.csv',
    'palembang': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/palembangMatchCategorized.csv'
    }

minioObjectName = {
    'jakarta': 'staging/csv/duckdb/JawaBarat/jakarta.csv',
    'bekasi': 'staging/csv/duckdb/JawaBarat/bekasi.csv',
    'depok': 'staging/csv/duckdb/JawaBarat/depok.csv',
    'balikpapan': 'staging/csv/duckdb/kalimantanTimur/balikpapan.csv',
    'makassar': 'staging/csv/duckdb/sulawesiSelatan/makassar.csv',
    'palembang': 'staging/csv/duckdb/sumateraSelatan/palembang.csv'
    }

MinIO Setup

In [3]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

duckdb Setup

In [4]:
con = duckdb.connect()
con.sql("INSTALL httpfs; LOAD httpfs;")

Minio Connection

In [5]:
con.sql(f"SET s3_endpoint = '{minioAuth['endpoint'].replace('http://', '')}';")
con.sql("SET s3_use_ssl = false;") 
con.sql(f"SET s3_access_key_id = '{minioAuth['accessKey']}';")
con.sql(f"SET s3_secret_access_key = '{minioAuth['secretKey']}';")
con.sql("SET s3_url_style = 'path';") # WAJIB untuk MinIO/Local

Full EndPoint URL

In [6]:
fullEndPointURL = {
    'jakarta': f"s3://{minioAuth['bucket']}/{minioObjectName['jakarta']}",
    'bekasi': f"s3://{minioAuth['bucket']}/{minioObjectName['bekasi']}",
    'depok': f"s3://{minioAuth['bucket']}/{minioObjectName['depok']}",
    'balikpapan':  f"s3://{minioAuth['bucket']}/{minioObjectName['balikpapan']}",
    'makassar': f"s3://{minioAuth['bucket']}/{minioObjectName['makassar']}",
    'palembang': f"s3://{minioAuth['bucket']}/{minioObjectName['palembang']}"
    }

In [7]:
loadJakarta = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['jakarta']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['jakarta']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadBekasi = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['bekasi']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['bekasi']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadDepok = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['depok']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['depok']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadBalikpapan = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['balikpapan']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['balikpapan']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadMakassar = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['makassar']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['makassar']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""
loadPalembang = f"""
    COPY 
        (
            SELECT * -- Ini berarti ambil SEMUA kolom dan SEMUA baris
            FROM 
                read_csv_auto('{localPath['palembang']}') -- Membaca data dari CSV lokal Anda
        ) 
    TO 
        '{fullEndPointURL['palembang']}'  -- Menulis hasilnya langsung ke MinIO
    (FORMAT 'PARQUET', 
     OVERWRITE TRUE);
"""

In [8]:
# --- 2. Jalankan Proses Migrasi ---
try:
    con.sql(loadJakarta)
    con.sql(loadBekasi)
    con.sql(loadDepok)
    con.sql(loadBalikpapan)
    con.sql(loadMakassar)
    con.sql(loadPalembang)
    
    print(f"\n✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO")

except Exception as e:
    print(f"\n❌ Terjadi error saat migrasi: {e}")


✅ Sukses! Data CSV telah dibaca, dikonversi ke Parquet, dan dikirim langsung ke MinIO


In [9]:
con.close()